<a href="https://colab.research.google.com/github/Pineappleaki/Pineappleaki/blob/main/python_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base Concept

taking insperation from this [article](https://towardsdatascience.com/how-to-build-your-own-chatbot-using-deep-learning-bb41f970e281)

The main idea to create a dataset to train the model on this will be called the intent file.
This intent file can be modified to make IANUA answer questions specifically that our users may ask such as game progression


In [ ]:
# Import module

import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
with open('intents.json') as f:
  data = json.load(f)

trn = []
trn_labels = []
labels = []
responses = []

for i in data['intents']:
  for j in i['patterns']:
    trn.append(j)
    trn_labels.append(i['tag'])
  responses.append(i['responses'])

  if i['tag'] not in labels:
    labels.append(i['tag'])

num_classes = len(labels)

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(trn_labels)
trn_labels = label_encoder.transform(trn_labels)

vocab_size = 1000
embedding_dim = 16
max_len = 30
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_token)
tokenizer.fit_on_texts(trn)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(trn)
p_sequences = pad_sequences(sequences, truncating = 'post', maxlen = max_len)


In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 9)                 153       
                                                        

In [ ]:
epochs = 750
history = model.fit(p_sequences, np.array(trn_labels), epochs=epochs)

Epoch 1/750
2/2 [==============================] - 1s 13ms/step - loss: 2.1968 - accuracy: 0.1892
Epoch 2/750
2/2 [==============================] - 0s 8ms/step - loss: 2.1951 - accuracy: 0.1892
Epoch 3/750
2/2 [==============================] - 0s 8ms/step - loss: 2.1935 - accuracy: 0.1892
Epoch 4/750
2/2 [==============================] - 0s 7ms/step - loss: 2.1930 - accuracy: 0.1892
Epoch 5/750
2/2 [==============================] - 0s 6ms/step - loss: 2.1923 - accuracy: 0.1892
Epoch 6/750
2/2 [==============================] - 0s 6ms/step - loss: 2.1913 - accuracy: 0.1892
Epoch 7/750
2/2 [==============================] - 0s 15ms/step - loss: 2.1905 - accuracy: 0.1892
Epoch 8/750
2/2 [==============================] - 0s 8ms/step - loss: 2.1893 - accuracy: 0.1892
Epoch 9/750
2/2 [==============================] - 0s 8ms/step - loss: 2.1881 - accuracy: 0.1892
Epoch 10/750
2/2 [==============================] - 0s 7ms/step - loss: 2.1874 - accuracy: 0.1892
Epoch 11/750
2/2 [=========

In [ ]:
# to save the trained model
model.save("chat_model")

import pickle
import random

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(label_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)


INFO:tensorflow:Assets written to: chat_model/assets


In [ ]:
def chat():
  model = keras.models.load_model('chat_model')

  # load tokenizer object
  with open('tokenizer.pickle', 'rb') as handle:
      tokenizer = pickle.load(handle)

  # load label encoder object
  with open('label_encoder.pickle', 'rb') as enc:
      label_encoder = pickle.load(enc)

  while True:
    print('User: ', end = "")
    inp = input()
    if inp.lower() == 'quit':
      break

    result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                          truncating='post', maxlen=max_len))
    tag = label_encoder.inverse_transform([np.argmax(result)])

    for i in data['intents']:
      if i['tag'] == tag:
        print(f'IANUA:',(np.random.choice(i['responses'])))

print('begin:')
chat()

begin:
User: Hello
IANUA: Hello
User: how are you
IANUA: none of your business
User: What cup am i
IANUA: My pleasure
User: 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelbase.py", line 729, in _input_request
    ident, reply = self.session.recv(self.stdin_socket, 0)
  File "/usr/local/lib/python3.7/dist-packages/jupyter_client/session.py", line 803, in recv
    msg_list = socket.recv_multipart(mode, copy=copy)
  File "/usr/local/lib/python3.7/dist-packages/zmq/sugar/socket.py", line 727, in recv_multipart
    parts = [self.recv(flags, copy=copy, track=track)]
  File "zmq/backend/cython/socket.pyx", line 803, in zmq.backend.cython.socket.Socket.recv
  File "zmq/backend/cython/socket.pyx", line 839, in zmq.backend.cython.socket.Socket.recv
  File "zmq/backend/cython/socket.pyx", line 188, in zmq.backend.cython.socket._recv_copy
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/li

TypeError: ignored

In [ ]:
print("q1: are you introverted")
input = input()

if input == '1':
  print('introverted')
else:
  print('extro')

q1: are you introverted


TypeError: ignored